In [1]:
# !wget https://huggingface.co/datasets/commonsense_qa/resolve/refs%2Fconvert%2Fparquet/default/train/0000.parquet -O commonsense-train.parquet

In [3]:
from tqdm import tqdm
import os
import json

In [2]:
import pandas as pd

df = pd.read_parquet('commonsense-train.parquet')

In [3]:
df.shape

(9741, 5)

In [4]:
df.iloc[0]

id                                   075e483d21c29a511267ef62bedc0461
question            The sanctions against the school were a punish...
question_concept                                            punishing
choices             {'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...
answerKey                                                           A
Name: 0, dtype: object

In [5]:
instructions = []
for k in range(len(df)):
    choices = df['choices'].iloc[k]
    choices = [f"{choices['label'][i]}. {choices['text'][i]}" for i in range(len(choices['text']))]
    choices = '\n'.join(choices)
    s = f"Jawab soalan berikut.\n{df['question'].iloc[k]}\n{choices}\nJawapan: {df['answerKey'].iloc[k]}"
    instructions.append(s)
len(instructions)

9741

In [6]:
import os
import openai

openai.api_type = "azure"
openai.api_base = "https://husein-ai2-aiservices.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = ''

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [7]:
!mkdir commonsense

mkdir: cannot create directory ‘commonsense’: File exists


In [8]:
for i in tqdm(range((len(instructions) // 3) * 0, (len(instructions) // 3) * 1, 1)):
    filename = f'commonsense/{i}.json'
    if os.path.exists(filename):
        continue
        
    r = []
    for k in range(1):
        
        try:
            message_text = [
                {"role":"user","content": f"{instructions[i]}\n\ngenerate similar malay questions included answers like above."},
            ]
            completion = openai.ChatCompletion.create(
              engine="gpt-4",
              messages = message_text,
              temperature=1.0,
              max_tokens=1024,
              top_p=0.95,
              frequency_penalty=0,
              presence_penalty=0,
              stop=None
            )
            r.append(completion.choices[0]['message']['content'])
            break
            

        except Exception as e:
            print(e)
        
    with open(filename, 'w') as fopen:
        json.dump(r, fopen)

  2%|▋                                     | 63/3247 [13:41<13:06:01, 14.81s/it]

The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766


  6%|██                                   | 186/3247 [44:32<11:46:39, 13.85s/it]

The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766


  6%|██▍                                   | 204/3247 [47:49<7:43:56,  9.15s/it]

...

 87%|█████████████████████████████▌    | 2828/3247 [10:32:37<1:31:44, 13.14s/it]

The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766


 93%|█████████████████████████████████▌  | 3023/3247 [11:14:29<54:56, 14.72s/it]

The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766


100%|████████████████████████████████████| 3247/3247 [12:07:19<00:00, 13.44s/it]


In [5]:
from glob import glob

files = glob('commonsense/*.json')
len(files)

9741

In [15]:
augmentation = []
for f in files:
    with open(f) as fopen:
        datas = json.load(fopen)
    for data in datas:
        splitted = data.split('\n\n')
        for s in splitted:
            try:
                a = s.split('Jawapan:')[1].strip()
                augmentation.append({
                    'question': s.split('Jawapan:')[0].strip(),
                    'answer': a,

                })
            except:
                pass
            
len(augmentation)

36332

In [17]:
with open('synthetic-commonsense.jsonl', 'w') as fopen:
    for a in augmentation:
        fopen.write(f'{json.dumps(a)}\n')